In [92]:
import math
import tkinter as tk
import pandas as pd

In [93]:
def nearest_tens(x_low, x_high):
    lower = math.floor(x_low / 10) * 10
    upper = math.ceil(x_high / 10) * 10
    if lower == x_low:
        lower -= 10
    if upper == x_high:
        upper += 10
    return lower, upper

In [94]:
def create_axes_with_ticks(canvas, width, height,x_low,x_high,y_low, y_high):
    # Margin for text and ticks to be fully visible
    m = 30 # margin
    w = width
    h = height

    x_l,x_h = nearest_tens(x_low,x_high)
    y_l,y_h = nearest_tens(y_low,y_high)

    x_range = x_h - x_l
    y_range = y_h - y_l


    # Draw X and Y axes
    axis_color = 'black'
    canvas.create_line(2*m , h-2*m , 2*m , m ,arrow=tk.LAST, fill=axis_color) # X-axis
    canvas.create_line(2*m , h-2*m , w - m, h-2*m ,  arrow=tk.LAST,fill=axis_color) # Y-axis

    # Ticks settings
    tick_length = 5
    number_of_xticks = 10
    number_of_yticks = 10
    gap_x = (w-4*m) / number_of_xticks
    gap_y = (h-4*m) / number_of_yticks

    gap_x_value = x_range/number_of_xticks
    gap_y_value = y_range/number_of_yticks
    # Drawing X-axis ticks and labels
    for i in range(number_of_xticks + 1):
        x =  2*m +i*gap_x
        canvas.create_line(x, h - 2*m, x, h - 2*m - tick_length, fill=axis_color)
        canvas.create_text(x, h - 2*m + tick_length, text=str(x_l + i*gap_x_value), anchor='n', font=('Arial', 8))

    # Drawing Y-axis ticks and labels
    for i in range(number_of_yticks + 1):
        y = h-2*m - i*gap_y
        canvas.create_line(2*m, y, 2*m + tick_length, y, fill=axis_color)
        canvas.create_text(2*m - tick_length, y, text=str(y_l+i*gap_y_value), anchor='e', font=('Arial', 8))


In [96]:

data, x_min,x_max,y_min,y_max,num_labels = get_data('data1.csv')
print ("x_min:",x_min,",x_max:",x_max,"y_min:",y_min,",y_max:",y_max)
data.head()

x_min: -52 ,x_max: 55 y_min: -55 ,y_max: 54


,x,y,label
0,-12,20,a
1,-23,23,a
2,50,-12,b
3,1,54,c
4,2,2,c


In [97]:
import math 

def draw_star(canvas, center_x, center_y, size, fill_color):
    """ 绘制一个空心五角星，并填充指定颜色。

    :param canvas: Tkinter Canvas对象
    :param center_x: 五角星中心的x坐标
    :param center_y: 五角星中心的y坐标
    :param size: 五角星的大小（从中心到顶点的距离）
    :param fill_color: 五角星填充颜色
    """
    points = []
    for i in range(5):
        # 计算外部顶点
        x = center_x + size * math.cos(2 * math.pi * i / 5 - math.pi / 10)
        y = center_y - size * math.sin(2 * math.pi * i / 5 - math.pi / 10)
        points.append((x, y))

        # 计算内部顶点
        x = center_x + size/2 * math.cos(2 * math.pi * i / 5 + math.pi / 10)
        y = center_y - size/2 * math.sin(2 * math.pi * i / 5 + math.pi / 10)
        points.append((x, y))

    canvas.create_polygon(points, outline='black', fill=fill_color)


In [103]:
def translate(x,y,x_low,x_high,y_low, y_high,width = 500,height = 500,margin = 30):
    x_l,x_h = nearest_tens(x_low,x_high)
    y_l,y_h = nearest_tens(y_low,y_high)

    x_range = x_h - x_l
    y_range = y_h - y_l

    x_pixel = (x - x_l)/x_range*(width-4*margin) + 2*margin
    y_pixel = height- 2*margin - (y - y_l)/y_range*(height-4*margin)

    return x_pixel,y_pixel


In [105]:

# Create Tkinter window
window = tk.Tk()
window.title("Axes with Ticks")

# Set window size
window.geometry('600x600')

# Create Canvas widget
width = 500
height = 500
canvas = tk.Canvas(window, width=width, height=height)
canvas.pack()

# Create axes with ticks
create_axes_with_ticks(canvas, width, height,x_min,x_max,y_min,y_max)

for index, row in data.iterrows():
    x, y = translate(row['x'],row['y'],x_min,x_max,y_min,y_max)
    draw_star(canvas,x,y,5,'red')

# Run the window's main loop
window.mainloop()
